# <div align="center">Rutile-Trace-Elements</div>
## <div align="center">Python code for visualizing detrital rutile trace element geochemistry data</div>
#### <div align="center">Megan Mueller</div>
##### <div align="center">Please cite the accompanying article under consideration in Geochronology:</div>
<div align="center">Mueller et al., https://doi.org/10.5194/egusphere-2023-1293 </div>



<br><div align="center">Version 1.1.2</div>

# Overview 
This notebook aims to produce various plots for visualizing trace element data from detrital rutile. 

### Initial Conditions
The notebook uses the Excel file with results from several other notebooks: corrected date and uncertainty from the Detrital-Common-Pb-Corrections notebook and percent concordant from the UPb-Plotter notebook.

The code uses colorblind-friendly color maps from Fabio Crameri (https://www.fabiocrameri.ch/colourmaps/)

### Organization
This notebook is divided into the following sections: 
1. [Data import and initial code](#Section1) (must run first) 
2. [TiO2 polymorphs](#Section2)
3. [Mafic vs pelitic protoliths](#Section3)
4. [Zr-in-rutile thermometry](#Section4)
5. [Low U rutile](#Section5)
6. [Export data](#Section6)

<a id='Section1'></a>
# 1. Import Data and Set-Up


### 1.1 Install Python Packages and Import Libraries


In [ ]:
### Install Python packages
# Run to see if installed, if not, will install
# Only need to install once

import importlib
from subprocess import run

def install_if_not_installed(package_name):
    try:
        importlib.import_module(package_name)
        print(f"Package {package_name} is already installed.")
    except ImportError:
        print(f"Installing package: {package_name}")
        run(['pip', 'install', package_name], check=True)
        print(f"Installed package: {package_name}")

install_if_not_installed("pandas") # Install pandas
install_if_not_installed("numpy") # Install numpy
install_if_not_installed("matplotlib") # Install matplotlib
install_if_not_installed("tabulate") # Install tabulate
install_if_not_installed("sympy") # Install sympy
install_if_not_installed("easygui") # Install easygui
install_if_not_installed("cmcrameri") # Install cmcrameri

In [ ]:
### Import necessary libraries
import os
import numpy as np
from sympy import symbols, Eq, solve, exp
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
from tabulate import tabulate
import seaborn as sns
import easygui
from cmcrameri import cm
%matplotlib inline
%matplotlib notebook

### 1.2 Input File Organization (Data Needed to Run)
In addition to trace element concentations, the following sections of code use the following data: 238/206 and 207/206 ratios, U (ppm), 207Pb-corrected age and uncertainty, and percent discordant and concordant. The UPbPlotter outut table should be used as the input file which has all of the relecant columns, or can load multiple Excel spreadsheets.

The cell below can be modified to run in one of two ways: (1) import one Excel file with trace element data and U-Pb data, or (2) import two Excel files with trace element and U-Pb data separately. If importing 2 Excel files, the code will combine into one file, so must have same number of rows and same grain in each row. The code will prompt the user to answer whether there are one or two files to import. Code will not continue without completing prompt.

Data needed to run:
- Grain information (i.e., grain name, sample name, stratigraphic position)
- Trace element concentrations as ppm
- U concentration as ppm
- 238/206 and 207/206 ratios and uncertainties
- 207Pb corrected date and uncertainty (from Detrital-Common-Pb-Corrections)
- Percent concordance and discordance (from UPb-Plotter)





In [ ]:
### Load Excel file with data into DataFrame

# Edit the following lines to direct to the Excel file with trace element data
folder_path1 = r'C:\Users\megan\Dropbox\CommonPbPython\Example-data'   # Update this with the actual path
file_name1 = 'ExampleData-RutileTraceElements-Muelleretal.xlsx'  # Update this with the actual first file name
sheet_name1 = 'Sheet1' # Sheet name in Excel file

excel_file_path1 = os.path.join(folder_path1, file_name1) # Combine folder path and file name using os.path.join
df1 = pd.read_excel(excel_file_path1, sheet_name=sheet_name1) # Load the first file into a DataFrame

user_input = input("\033[1mWould you like to import one or two Excel files (1/2): \033[0m\n type '1' or '2' not 'one' or 'two' \n If 1, that means that all TE and U-Pb data are together in one Excel file. \n If 2, can import two Excel files: one with TE and another with U-Pb \n")
if user_input.lower() == '1':
    df = df1
    print("\033[1m1 Excel file loaded\033[0m")
elif user_input == '2':
    # Specify the folder and file names for the second file that contains U-Pb data if not in first file
    folder_path2 = r'path\to\second\folder'  # Update this with the actual path
    file_name2 = 'OutputFile_207Correction_v1.xlsx'  # Update this with the actual second file name
    sheet_name2 = 'Sheet1' # Update this with the actual sheet name of the second file

    excel_file_path2 = os.path.join(folder_path2, file_name2) # Combine folder path and file name using os.path.join
    df2 = pd.read_excel(excel_file_path2, sheet_name=sheet_name2) # Load the second file into a DataFrame

    df = pd.concat([df1, df2], ignore_index=True)     # Combine DataFrames
    print("\033[1m2 Excel files loaded.\033[0m")
else:
    print("\033[1mOops. Invalid input. Please enter '1' or '2'.\033[0m")
    

# Display the combined DataFrame
pd.set_option('display.max_columns', None)
df.head(10)  # display first 10 rows of the combined DataFrame



<a id='DefineVariables'></a>
### 1.3 Define Variables
Edit DataFrame column header names as needed. Prompts user to answer whether input uncertainties are 1s absolute or 2s absolute. If 1s, will convert to 2s.


In [ ]:
### Edit this section to add/remove columns to include as variables
# Everything here will be used in subsequent code
# Edit if column names are different than below. See column names in above cell output (or, run df.head(15) )

# Define variables by column headers
data_dict = df.to_dict(orient='series')

# Grain Information
# Load these if want to explore them in plots (i.e., color grains by Stratigraphic Position)
StratOrder = df['Stratigraphic Position']
GrainID = df['Grain_ID']
RunID = df['Analytical Run'] # not used, but provided as example
SampleID = df['Sample_ID'] # not used, but provided as example

# Trace elements
# Edit to include all elements measured
V = df['V'].copy()
Cr = df['Cr'].copy()
Zr = df['Zr'].copy()
Nb = df['Nb'].copy()
Ta = df['Ta'].copy()
U = df['Approx_U_PPM_mean'].copy()

# U-Pb data: Initial measured and calculated values
# Can use 207Pb-corrected output table as input file
i386 = df['Final U238/Pb206_mean'].copy()
i386err = df['Final U238/Pb206_2SE(prop)'].copy() # absolute
i76 = df['Final Pb207/Pb206_mean'].copy()
i76err = df['Final Pb207/Pb206_2SE(prop)'].copy() # absolute
XsErrPerc = i386err / i386 * 100 # define percent uncertainty
YsErrPerc = i76err / i76 * 100 # define percent uncertainty

# Pb Corrected Date
DateCorr = df['It5_**Age207c**'].copy() # 207Pb-corrected age (Ma) from final iteration
DateCorr_err = df['It5_**Age207cErr**'].copy() # 207Pb-corrected age uncertainty (2s absolute) from final iteration

# Concordance and Discordance
PercDisc = df['SK_percent_discordant2'].copy() # Percent discordant
PercConc = df['SK_percent_concordant2'].copy()

# Uncorrected U-Pb points used in subsequent plots
Xs386 = i386 # avoid overwriting
Ys76 = i76
Xs386[Xs386 == 0] = np.nan # replace 0.0 values with NaN
Ys76[Ys76 == 0] = np.nan


# Check if uncertainties are 2s abs or 1s abs
user_input = input("\033[1mAre input file uncertainties 1-sigma absolute or 2-sigma absolute? \033[0m\n Type '1' or '2' (not 'one' or 'two') \n If 2, no calculations will be performed \n If 1, input uncertainties will be multiplied by 2 \n")
if user_input.lower() == '2':
    print("\033[1mInput uncertainties are 2s\033[0m")
elif user_input == '1':
    # Will run the following code if input uncertainties are 1s (abs) and not 2s (abs)
    i386err = i386err * 2 # 2s absolute
    i76err = i76err * 2 # 2s absolute
    XsErrPerc = XsErrPerc * 2 # 2s percent
    YsErrPerc = YsErrPerc * 2 # 2s percent
    DateCorr_err = DateCorr_err * 2 # 207Pb-corrected age uncertainty (absolute)
    print('\033[1mReminder:\033[0m Input uncertainties are 1s and were multiplied by 2. Subsequent calculations and outputs will be at 2s level')
else:
    print("\033[1mOops. Invalid input. Please enter '1' or '2'.\033[0m")

<a id='ConcordiaPoints'></a>Define constants and initial calculations. Define age points along concordia.

In [ ]:
# Constants and initial calculations

# Define constants
instUTh = 1; # instrument-specific U/Th, should be =1 and already corrected during data reduction
rat85 = 137.88; # 238U/235U ratio. 137.88 from Steiger and Jäger (1977). 137.818 from Hiess et al. (2012). 
rat58 = 0.007252683; # 1/137.88 constant 235/238 U ratio
lambda238 = 1.55125E-10; # U238 decay constant from Faure (1986)
lambda235 = 9.8485E-10; # U235 decay constant from Faure (1986)
lambda232 = 4.9475E-11; # 232Th decay constant

# Set up concordia curve for subsequent plots
t = np.linspace(1, 6000000000, 1000)  # 1000 points between 1 and 6000000000, time in yrs
X86 = 1 / (np.exp(lambda238 * t) - 1)  # 238/206 ratio over time
Y76 = (1 / rat85) * (np.exp(lambda235 * t) - 1) / (np.exp(lambda238 * t) - 1)  # 207/206 ratio over time

# Mark some points along the line at specified times
# Can edit to change which Date markers are displayed on concordia curve
tMyr = np.array([50e6, 75e6, 100e6, 200e6, 350e6, 500e6, 750e6, 1000e6, 2000e6, 3000e6, 4000e6])
X86tMyr = 1 / (np.exp(lambda238 * tMyr) - 1)  # 238/206 ratio at specified time
Y76tMyr = (1 / rat85) * (np.exp(lambda235 * tMyr) - 1) / (np.exp(lambda238 * tMyr) - 1)  # 207/206 ratio at specified time


# Initial calculations
logV = np.log10(V)
logCr = np.log10(Cr)
logZr = np.log10(Zr)
CrNb = Cr / Nb
logCrNb = np.log10(Cr / Nb)
NbTa = Nb / Ta

<a id='Section2'></a>

# 2. TiO2 Polymorphs

This section uses the trace element data to discriminate TiO2 polymorphs following Triebold et al. (2011). The second section replicates the first's plot with the inclusion of the TiO2 polymorph dataset of Triebold et al. (2011).

### 2.1 TiO2 Polymorph Plots using Cr, Zr, V

In [ ]:
### TiO2 Polymorphs
# Define the color of points in plot, suggested Stratigraphic Position
# Edit as needed
color_variable = df['Stratigraphic Position'].copy() # Variable to color points. Change to desired input table heading
color_label = 'Stratigraphic Position' # Color bar label
color_variable = pd.to_numeric(color_variable, errors='coerce')
palette = 'gist_earth' #cm.batlowW, 'cubehelix'

# Calculate rounded min and max values
min_logV = np.floor(np.min(logV) / 0.5) * 0.5
max_logV = np.ceil(np.max(logV) / 0.5) * 0.5
min_logCr = np.floor(np.min(logCr) / 0.5) * 0.5
max_logCr = np.ceil(np.max(logCr) / 0.5) * 0.5
min_logZr = np.floor(np.min(logZr) / 0.5) * 0.5
max_logZr = np.ceil(np.max(logZr) / 0.5) * 0.5

# log(V-ppm) vs log(Cr-ppm)
plt.close()
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 10))

s1 = sns.scatterplot(x=logV, y=logCr, hue=color_variable, palette=palette, edgecolor='k', s=50, legend=False, ax=axes[0])
sm = plt.cm.ScalarMappable(cmap=palette, norm=plt.Normalize(vmin=color_variable.min(), vmax=color_variable.max()))
sm.set_array([])  # An empty array is required
colorbar = plt.colorbar(mappable=sm, ax=axes[0])
colorbar.set_label(color_label, rotation=270, labelpad=15)
colorbar.set_ticks(color_variable.unique())  # Set ticks to 'StratPos' values
axes[0].set_title('log(V-ppm) vs log(Cr-ppm) after Triebold et al. 2011')
axes[0].set_xlabel('log(V [ppm])')
axes[0].set_ylabel('log(Cr [ppm])')
axes[0].set_xlim(min_logV, max_logV)
axes[0].set_ylim(min_logCr, max_logCr)
axes[0].grid(False)

# log(Zr) vs log(V)
s2 = sns.scatterplot(x=logZr, y=logV, hue=color_variable, palette=palette, edgecolor='k', s=50, legend=False, ax=axes[1])
sm = plt.cm.ScalarMappable(cmap=palette, norm=plt.Normalize(vmin=color_variable.min(), vmax=color_variable.max()))
sm.set_array([])  # An empty array is required
colorbar = plt.colorbar(mappable=sm, ax=axes[1])
colorbar.set_label(color_label, rotation=270, labelpad=15)
colorbar.set_ticks(color_variable.unique())  # Set ticks to 'StratPos' values
axes[1].set_title('log(Zr-ppm) vs log(V-ppm) after Triebold et al. 2011')
axes[1].set_xlabel('log(Zr [ppm])')
axes[1].set_ylabel('log(V [ppm])')
axes[1].set_xlim(min_logZr, max_logZr)
axes[1].set_ylim(min_logV, max_logV)
axes[1].grid(False)

plt.tight_layout()
plt.show()



In [ ]:
### Run to save figure
# Run immediately after generating figure or might save a different figure instead
user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "TiO2_polymorphs_v1.pdf"),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        fig.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

### 2.2 Plot with Polymorph Dataset
The following cells repeats the above plot with the addition of the Triebold et al. (2011) TiO2 polymorphs dataset. Grains in this dataset were classified using Raman spectroscopy and give a general sense for where rutile should plot. 1. Load Triebold et al. 2011 dataset; 2. plot; 3. save figure.

*Triebold, S., Luvizotto, G. L., Tolosana-Delgado, R., Zack, T., & von Eynatten, H. (2011). Discrimination of TiO2 polymorphs in sedimentary and metamorphic rocks. Contributions to Mineralogy and Petrology, 161(4), 581–596. https://doi.org/10.1007/s00410-010-0551-x*


In [ ]:
### Plot TiO2 polymorphs with Triebold et al. 2011 dataset

# Import Triebold et al. 2011 anatase, brookite, rutile dataset
# Edit the following lines to direct to the Excel file with trace element data
folder_path_Triebold = r'C:\Users\megan\Dropbox\CommonPbPython\Example-data'  # Update this with the actual path
file_name_Triebold = 'Triebold_etal_2011_Data.xlsx'  # Update this with the actual file name
sheet_name_Triebold = 'Combined' # Sheet name in Excel file, 'Combined' is the combined supplementary data tables OM1 & OM2

excel_file_path_Triebold = os.path.join(folder_path_Triebold, file_name_Triebold) # Combine folder path and file name using os.path.join
df_Triebold = pd.read_excel(excel_file_path_Triebold, sheet_name=sheet_name_Triebold) # Load the first file into a DataFrame

logV_Triebold = df_Triebold['log(V)']
logCr_Triebold = df_Triebold['log(Cr)']
logZr_Triebold = df_Triebold['log(Zr)']

# Display the combined DataFrame
pd.set_option('display.max_columns', None)
df_Triebold.head(10)  # display first 10 rows of the combined DataFrame


In [ ]:
# Define the color of points in plot, suggested Stratigraphic Position
# Edit as needed
color_variable = df['Stratigraphic Position'].copy() # Variable to color points by. Change as needed
color_label = 'Stratigraphic Position' # Color bar label
color_variable = pd.to_numeric(color_variable, errors='coerce')
palette = 'gist_earth' #cm.batlowW, 'cubehelix'

# Set markers for Triebold et al 2011 data set
df_Triebold['Marker'] = np.select(
    [df_Triebold['Raman Mineral ID'] == 'Anatase',
     df_Triebold['Raman Mineral ID'] == 'Brookite',
     df_Triebold['Raman Mineral ID'] == 'Rutile'],
    ['*', 'o', 'x'],
    default='.'
)

# Calculate rounded min and max values
min_logV = np.floor(min(logV.min(), logV_Triebold.min()) / 0.5) * 0.5
max_logV = np.ceil(max(logV.max(), logV_Triebold.max()) / 0.5) * 0.5
min_logCr = np.floor(min(logCr.min(), logCr_Triebold.min()) / 0.5) * 0.5
max_logCr = np.ceil(max(logCr.max(), logCr_Triebold.max()) / 0.5) * 0.5
min_logZr = np.floor(min(logZr.min(), logZr_Triebold.min()) / 0.5) * 0.5
max_logZr = np.ceil(max(logZr.max(), logZr_Triebold.max()) / 0.5) * 0.5

# log(V-ppm) vs log(Cr-ppm)
plt.close()
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 10))

s1 = sns.scatterplot(x=logV, y=logCr, hue=color_variable, palette=palette, edgecolor='k', s=50, legend=False, ax=axes[0], label='This study', zorder=2)
sm = plt.cm.ScalarMappable(cmap=palette, norm=plt.Normalize(vmin=color_variable.min(), vmax=color_variable.max()))
sm.set_array([])  # An empty array is required
colorbar = plt.colorbar(mappable=sm, ax=axes[0])
colorbar.set_label(color_label, rotation=270, labelpad=15)
colorbar.set_ticks(color_variable.unique())  # Set ticks to 'StratPos' values
#axes[0].set_title('log(V-ppm) vs log(Cr-ppm) after Triebold et al. 2011')
axes[0].set_xlabel('log(V [ppm])')
axes[0].set_ylabel('log(Cr [ppm])')
axes[0].set_xlim(min_logV, max_logV)
axes[0].set_ylim(min_logCr, max_logCr)
axes[0].grid(False)

markers = {'Anatase': '+', 'Brookite': 'x', 'Rutile': '1'}
alpha_values = {'Anatase': 0.3, 'Brookite': 0.6, 'Rutile': 0.9}  # Adjust alpha values for each category
for category, marker in markers.items():
    subset = df_Triebold[df_Triebold['Raman Mineral ID'] == category]
    axes[0].scatter(subset['log(V)'], subset['log(Cr)'],
                    marker=marker, color='gray', s=50, alpha=alpha_values[category], label=f'{category} (Triebold et al., 2011)', zorder=1)

# Legend    
handles, labels = axes[0].get_legend_handles_labels()
legend = axes[0].legend(handles, labels, loc='best', fancybox=True, shadow=False, ncol=1)   


# log(Zr) vs log(V)
s2 = sns.scatterplot(x=logZr, y=logV, hue=color_variable, palette=palette, edgecolor='k', s=50, legend=False, ax=axes[1], label='This study', zorder=2)
sm = plt.cm.ScalarMappable(cmap=palette, norm=plt.Normalize(vmin=color_variable.min(), vmax=color_variable.max()))
sm.set_array([])  # An empty array is required
colorbar = plt.colorbar(mappable=sm, ax=axes[1])
colorbar.set_label(color_label, rotation=270, labelpad=15)
colorbar.set_ticks(color_variable.unique())  # Set ticks to 'StratPos' values
#axes[1].set_title('log(Zr-ppm) vs log(V-ppm) after Triebold et al. 2011')
axes[1].set_xlabel('log(Zr [ppm])')
axes[1].set_ylabel('log(V [ppm])')
axes[1].set_xlim(min_logZr, max_logZr)
axes[1].set_ylim(min_logV, max_logV)
axes[1].grid(False)

for category, marker in markers.items():
    subset = df_Triebold[df_Triebold['Raman Mineral ID'] == category]
    axes[1].scatter(subset['log(Zr)'], subset['log(V)'],
                    marker=marker, color='gray', s=50, alpha=alpha_values[category], label=f'{category} (Triebold et al., 2011)', zorder=1)

# Legend       
handles, labels = axes[1].get_legend_handles_labels()
#legend = axes[1].legend(handles, labels, loc='best', fancybox=True, shadow=False, ncol=1)   

plt.tight_layout()
plt.show()




In [ ]:
### Run to save figure
user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "TiO2_polymorphs_Triebold2012Data_v1.pdf"),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        fig.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

<a id='Section3'></a>

# 3. Mafic vs Pelitic Protoliths

The following section uses the trace element data to explore whether detrital rutile were derived from metamafic or metapelitic sources.

## 3.1 Protolith Classification
This section defines whether grains plot in the mafic or pelitic discrimination field relative to the field boundary from Triebold et al. (2012). Then, the number of mafic, pelitic or No TREE points are calculated for (1) full dataset, (2) grains in concordia diagram (same as 1 if all grains have U-Pb and TREE), (3) grains below uncertainty threshold.

<a id='UPbUncertainty'></a>The uncertainty threshold is set at a given percent uncertainty for both 238/206 and 207/206 ratios, where points above the threshold are excluded (change % uncertainty threshold in code below, as needed).

Note: the U-Pb ratio uncertainty filter defined here is used in several subsequent plots


In [ ]:
### Define the mafic and pelitic categories based on the discrimination line from Triebold et al., 2012: Cr = 5 * (Nb - 500)

# The third subplot only includes uncorrected U-Pb data with 238/206 and 207/206 uncertainties below specified threshold
uncertainty_threshold = 20 # Uncertainty threshold in %, excludes U-Pb ratios above this threshold


# Drop columns to write new data
columns_to_drop = ['MPClassification','MPValue_Triebold2012'] # Columns to drop
for column in columns_to_drop:# Check if each column exists before dropping
    if column in df.columns:
        df = df.drop(columns=[column])
        print(f"Column '{column}' dropped, to be replaced in DataFrame.")
    else:
        print(f"Column '{column}' does not exist in the DataFrame, does not need to be dropped.")

# Using the Triebold 2012 discimination line Cr = 5 * (Nb - 500), classify grains as "mafic" or "pelitic"
line_Triebold2012 = 5 * (Nb - 500)
df['Cr_expected'] = 5 * (Nb - 500) # For a given measured Nb, calculate Cr value at discrimination line
df['MPClassification'] = np.select(
    [
        df['Cr_expected'] > Cr,
        df['Cr_expected'] < Cr,
        pd.isnull(df['Cr_expected'])
    ],
    [
        'pelitic',
        'mafic',
        'NoProtolith'
    ],
    default='NoProtolith'
)
print('\n',df['MPClassification'].value_counts())
MPValue_Triebold2012 = (5 * (Nb - 500)) - Cr # Mafic if > 0, pelitic if < 0
df['MPValue_Triebold2012'] = MPValue_Triebold2012


# Count number of grains in each class
mafic_points = (df['MPClassification'] == 'mafic').sum()
pelitic_points = (df['MPClassification'] == 'pelitic').sum()
total_points = mafic_points + pelitic_points
print(f"Number of points with 'MPClassification' = 'mafic': {mafic_points}")
print(f"Number of points with 'MPClassification' = 'pelitic': {pelitic_points}")
print(f"Total number of points: {total_points}")


# Count number of mafic, pelitic, NoTREE points in concordia diagram
# Totals may differ if not all grains have U-Pb and TREE data
mafic_concordia_points = (df['MPClassification'] == 'mafic') & (~pd.Series(Xs386).isna()) & (~pd.Series(Ys76).isna())
mafic_concordia_points_sum = mafic_concordia_points.sum()
pelitic_concordia_points = (df['MPClassification'] == 'pelitic') & (~pd.Series(Xs386).isna()) & (~pd.Series(Ys76).isna())
pelitic_concordia_points_sum = pelitic_concordia_points.sum()
NoTREE_concordia_points = (df['MPClassification'] == 'nan') & (~pd.Series(Xs386).isna()) & (~pd.Series(Ys76).isna())
NoTREE_concordia_points_sum = NoTREE_concordia_points.sum()
total_concordia = mafic_concordia_points_sum + pelitic_concordia_points_sum + NoTREE_concordia_points_sum
#total_concordia2 = np.count_nonzero(~np.isnan(Xs386)) # number of U-Pb points not NaN, should be same as total_concordia
print("\nMafic points in concordia diagram:", mafic_concordia_points.sum())
print("Pelitic points in concordia diagram:", pelitic_concordia_points.sum())
print('NoTREE points in concordia diagram:', NoTREE_concordia_points.sum())
print('Total points in concordia diagram:', total_concordia)


# Count number of mafic, pelitic, NoTREE points in concordia diagram BELOW uncertainty threshold
# Uncertainty threshold mask: Boolean indexing to NOT select points with uncertainties greater than the uncertainty threshold (i.e., 20%)
mask = ~(XsErrPerc > uncertainty_threshold) & ~(YsErrPerc > uncertainty_threshold) & (~pd.Series(Xs386).isna()) & (~pd.Series(Ys76).isna()) # points below threshold and not NaN
mask_sum = mask.sum() # number of points below uncertainty threshold
nonmask_sum = len(mask) - mask.sum() # number of points above uncertainty threshold

mafic_concordia_points_uncertainty = (df['MPClassification'] == 'mafic') & mask
mafic_concordia_points_uncertainty_sum = mafic_concordia_points_uncertainty.sum()
pelitic_concordia_points_uncertainty = (df['MPClassification'] == 'pelitic') & mask
pelitic_concordia_points_uncertainty_sum = pelitic_concordia_points_uncertainty.sum()
NoTREE_concordia_points_uncertainty = (df['MPClassification'] == 'nan') & mask
NoTREE_concordia_points_uncertainty_sum = NoTREE_concordia_points_uncertainty.sum()
total_concordia_uncertainty = mafic_concordia_points_uncertainty_sum + pelitic_concordia_points_uncertainty_sum + NoTREE_concordia_points_uncertainty_sum

print("\nMafic points in concordia diagram below uncertaity threshold:", mafic_concordia_points_uncertainty_sum)
print("Pelitic points in concordia diagram below uncertaity threshold:", pelitic_concordia_points_uncertainty_sum)
print("NoTREE points in concordia diagram below uncertaity threshold:", NoTREE_concordia_points_uncertainty_sum)
print('Total points in concordia diagram below uncertaity threshold:', total_concordia_uncertainty)

print('\n\nNote: if not all grains have U-Pb and TREE, totals may differ')

# Calculate the Triebold et al 2007 discrimination line: log10(Cr/Nb) = 0 (Cr=Nb) 
line_values = np.linspace(0, max(Nb), 100)

In [ ]:
# Run to view updated DataFrame

# New columns added to right
# 'Cr_expected' - For a given measured Nb, calculate Cr value at discrimination line. Measured Cr values above or below determines protolith classification
# MPClassification - classified as mafic or pelitic or NoTemp
# MPValue_Triebold2012 -  (5 * (Nb - 500)) - Cr ... Mafic if > 0, pelitic if < 0. Used in below plots

df.head(15)


## 3.2 Protolith Discrimination Plots
The following cell creates 3 subplots: (1) Cr vs Nb plot to discriminate mafic and pelitic sources. Grains are classified as mafic or pelitic following the Triebold et al., 2012 line. (2) Uncorrected U-Pb data on Tera-Wasserburg diagram. Analyses are colored by mafic/pelitic/NoTREE classification. The concordia age markers are defined in [Section 1](#ConcordiaPoints). (3) Subplot 2 only U-Pb points are filtered by a U-Pb ratio 'uncertainty_threshold' set in [Section 3](#UPbUncertainty) (i.e, 20% uncertainty on 238/206 and 207/206 ratios).

In [ ]:
plt.close()

print('\n\033[1mNote: Number mafic, pelitic points may differ between plots due to grains having only U-Pb or TREE data\033[0m\nTotals reflect number of points with TREE and U-Pb in respective plot')

fig, axes = plt.subplots(nrows=3, figsize=(8, 15))

### Plot in subplots
### Subplot 1

axes[0].scatter(Nb[(df['MPClassification'] == 'mafic')], Cr[(df['MPClassification'] == 'mafic')], s=25, color='green', edgecolor='black', linewidth=0.5, label=f'Mafic (n={mafic_points}/{total_points})')
axes[0].scatter(Nb[(df['MPClassification'] == 'pelitic')], Cr[(df['MPClassification'] == 'pelitic')], s=25, color='orange', edgecolor='black', linewidth=0.5, label=f'Pelitic (n={pelitic_points}/{total_points})')

# Plot the discrimination field lines
axes[0].plot(Nb, line_Triebold2012, color='black', linestyle='-', label='Cr = 5 * (Nb - 500)\nTriebold et al., 2012')
axes[0].axvline(x=800, color='black', linestyle='--', label='Cr = 800\nMeinhold et al., 2008')
axes[0].plot(line_values, line_values, linestyle='-.', color='black', label='Cr = Nb\nTriebold et al., 2007')

# Customize the plot
#plt.title('Mafic vs Pelitic Protoliths')
axes[0].set_xlabel('Nb (ppm)')
axes[0].set_ylabel('Cr (ppm)')
axes[0].set_title('Cr vs Nb Discrimination Diagram')
axes[0].legend()

# Set the axis limits
# Calculate the rounded-up maximum value for both axes
max_Nb_rounded = np.ceil(np.max(Nb) / 100) * 100
max_Cr_rounded = np.ceil(np.max(Cr) / 100) * 100
axes[0].set_xlim(0, max_Nb_rounded)
axes[0].set_ylim(0, max_Cr_rounded)


### Subplot 2
# Plot concordia diagram in Tera-Wasserburg space
axes[1].scatter(X86tMyr, Y76tMyr, s=10, edgecolors='black', facecolors='black', linewidth=0.5, zorder=2, label='Concordia Points')  # Mark specified points along concordia in black
axes[1].plot(X86, Y76, color='red', zorder=1, label='Concordia Curve')  # Plot concordia
for i, age in enumerate(tMyr / 1e6):
    axes[1].text(X86tMyr[i] - 0.35, Y76tMyr[i] - 0.04, f'{age:.0f}', fontsize=8, color='black', ha='right', va='bottom')

# Scatter plot for mafic points
axes[1].scatter(Xs386[df['MPClassification'] == 'mafic'], Ys76[df['MPClassification'] == 'mafic'],
                label='Mafic', s=25, c='green', edgecolors='black', linewidth=0.5, zorder=5)

# Scatter plot for pelitic points
axes[1].scatter(Xs386[df['MPClassification'] == 'pelitic'], Ys76[df['MPClassification'] == 'pelitic'],
                label='Pelitic', s=25, c='orange', edgecolors='black', linewidth=0.5, zorder=4)

# Scatter plot for "No TREE" points
axes[1].scatter(Xs386[df['MPClassification'] == 'NoTemp'], Ys76[df['MPClassification'] == 'NoTemp'],
                label='No TREE', s=15, c='white', edgecolors='black', linewidth=0.5, zorder=3)

axes[1].set_xlabel(r'$^{238}$U/$^{206}$Pb')
axes[1].set_ylabel(r'$^{207}$Pb/$^{206}$Pb')
axes[1].set_title('Uncorrected U-Pb Data')
legend_labels = {'mafic': f'Mafic (n={mafic_concordia_points_sum}/{total_concordia})', 
                 'pelitic': f'Pelitic (n={pelitic_concordia_points_sum}/{total_concordia})', 
                 'No TREE': f'No TREE (n={NoTREE_concordia_points_sum}/{total_concordia})'}
legend_handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=6, markeredgecolor='black', label=label)
    for color, label in zip(['green', 'orange', 'white'], legend_labels.values())
]
axes[1].legend(handles=legend_handles, loc='upper right')

# Set x and y-axis limits for the second subplot
axes[1].set_xlim([-5, 100])  # Replace with your desired values
axes[1].set_ylim([-0.05, 2])  # Replace with your desired values


### Subplot 3
# U-Pb uncertainty threshold applied
axes[2].scatter(X86tMyr, Y76tMyr, s=10, edgecolors='black', facecolors='black', linewidth=0.5, zorder=2)  # Mark specified points along concordia in black
axes[2].plot(X86, Y76, color='red', zorder=1)  # Plot concordia
for i, age in enumerate(tMyr / 1e6): # plot text labels on concordia
    axes[2].text(X86tMyr[i]- 0.35, Y76tMyr[i] - 0.04, f'{age:.0f}', fontsize=8, color='black', ha='right', va='bottom')

# Scatter plot for mafic points below uncertainty threshold
axes[2].scatter(Xs386[(mask) & (df['MPClassification'] == 'mafic')], Ys76[(mask) & (df['MPClassification'] == 'mafic')],
                label='Mafic', s=25, c='green', edgecolors='black', linewidth=0.5, zorder=5)

# Scatter plot for pelitic points below uncertainty threshold
axes[2].scatter(Xs386[(mask) & (df['MPClassification'] == 'pelitic')], Ys76[(mask) & (df['MPClassification'] == 'pelitic')],
                label='Pelitic', s=25, c='orange', edgecolors='black', linewidth=0.5, zorder=4)

# Scatter plot for "No TREE" points below uncertainty threshold
axes[2].scatter(Xs386[(mask) & (df['MPClassification'] == 'NoTemp')], Ys76[(mask) & (df['MPClassification'] == 'NoTemp')],
                label='No TREE', s=15, c='white', edgecolors='black', linewidth=0.5, zorder=3)


axes[2].set_xlim([-5, 100])  # Replace with desired values
axes[2].set_ylim([-0.05, 1.2])  # Replace with desired values
axes[2].set_xlabel(r'$^{238}$U/$^{206}$Pb')
axes[2].set_ylabel(r'$^{207}$Pb/$^{206}$Pb')
axes[2].set_title(f'Uncorrected U-Pb Data with Uncertainty Filter ({uncertainty_threshold}%)')
legend_labels = {'mafic': f'Mafic (n={mafic_concordia_points_uncertainty_sum}/{total_concordia_uncertainty})', 
                 'pelitic': f'Pelitic (n={pelitic_concordia_points_uncertainty_sum}/{total_concordia_uncertainty})', 
                 'No TREE': f'No TREE (n={NoTREE_concordia_points_uncertainty_sum}/{total_concordia_uncertainty})'}
legend_handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=6, markeredgecolor='black', label=label)
    for color, label in zip(['green', 'orange', 'white'], legend_labels.values())
]
axes[2].legend(handles=legend_handles, loc='upper right')

axes[2].grid(False)


# Show the plot
plt.grid(False)
plt.show()


In [ ]:
### Run to save figure

user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "MaficPelitic_v1.pdf"),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        fig.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

## 3.3 Protolith vs Corrected Date
The cell below plots the mafic-pelitic classification (Triebold et al., 2012 discrimination field) versus corrected U-Pb date. Update the corrected date variable with the correct column name in the DataFrame ('DateCorr' in [Section 1](#DefineVariables)). Cannot be run without running first cell in [protolith section](#Section3)... U-Pb ratio 'uncertainty_threshold' set in [Section 3](#UPbUncertainty) (i.e, 20% uncertainty on 238/206 and 207/206 ratios).

The Cr and Nb values are transformed around Cr = 5*(Nb-500), so that above 1 is mafic and below 1 is pelitic. This calculation is performed in [Section 3.1](#Section3). The absolute values along y-axis do not have much meaning, but the plots provide a sense for how protoliths are distributed across date.


In [ ]:
### Plot Mafic-Pelitic vs Corrected Date
# y-axis is essentially distance of point from Triebold 2012 discrimination line. Defined as 'MPValue_Triebold2012' in Section 3 cell 1.
# no uncertainty filter applied

filter1 = ~( (DateCorr == 0) | DateCorr.isna() | df['MPClassification'].isna() ) # filter out values = 0 or NaN

# plot M-P by date
plt.close()
fig, axes = plt.subplots(nrows=2, figsize=(8, 12))
# Define the colors based on the mafic-pelitic classification of Triebold et al. 2012
#MPcolors = ['green' if y < 0 else 'orange' for y in MPValue_Triebold2012[filter1]]
# Scatter plot with specified colors
#plt.scatter(DateCorr[filter1], (line_Triebold2012/Cr)[filter1], c=MPcolors, edgecolors = "black", linewidth = 0.5)

# Scatter plot for mafic points
axes[0].scatter(DateCorr[(filter1) & (df['MPClassification'] == 'mafic')], (line_Triebold2012/Cr)[(filter1) & (df['MPClassification'] == 'mafic')],
                label='Mafic', s=25, c='green', edgecolors='black', linewidth=0.5, zorder=5)

# Scatter plot for pelitic points
axes[0].scatter(DateCorr[(filter1) & (df['MPClassification'] == 'pelitic')], (line_Triebold2012/Cr)[(filter1) & (df['MPClassification'] == 'pelitic')],
                label='Pelitic', s=25, c='orange', edgecolors='black', linewidth=0.5, zorder=4)


axes[0].axhline(y=1, color='black', linestyle='--', linewidth=1) # moderate vs high Temp line
axes[0].set_yscale('log')
axes[0].set_xlim(0, 1000)
axes[0].set_xlabel('Corrected Date (Ma)')
axes[0].set_ylabel('Mafic vs. Pelitic')
axes[0].set_title('Protolith vs Corrected Date')


# Scatter plot for mafic points
axes[1].scatter(DateCorr[(mask) & (filter1) & (df['MPClassification'] == 'mafic')], (line_Triebold2012/Cr)[(mask) & (filter1) & (df['MPClassification'] == 'mafic')],
                label='Mafic', s=25, c='green', edgecolors='black', linewidth=0.5, zorder=5)

# Scatter plot for pelitic points
axes[1].scatter(DateCorr[(mask) & (filter1) & (df['MPClassification'] == 'pelitic')], (line_Triebold2012/Cr)[(mask) & (filter1) & (df['MPClassification'] == 'pelitic')],
                label='Pelitic', s=25, c='orange', edgecolors='black', linewidth=0.5, zorder=4)


axes[1].axhline(y=1, color='black', linestyle='--', linewidth=1) # moderate vs high Temp line
axes[1].set_yscale('log')
axes[1].set_xlim(0, 1000)
axes[1].set_xlabel('Corrected Date (Ma)')
axes[1].set_ylabel('Mafic vs. Pelitic')
axes[1].set_title(f'Protolith vs Corrected Dates Below Uncertainty Threshold ({uncertainty_threshold}%)')

plt.show()

In [ ]:
### Run to save figure

user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "MaficPeliticVsDate_v1.pdf"),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        fig.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

## 3.3 Protolith vs Zr-in-rutile temperature

The cell below plots the mafic-pelitic classification (Triebold et al., 2012 discrimination field) versus Zr-in-rutile temperature. 

Cannot be run without running (1) first cell in [protolith section](#Section3)... U-Pb ratio 'uncertainty_threshold' set in [Section 3](#UPbUncertainty) (i.e, 20% uncertainty on 238/206 and 207/206 ratios), and (2) Zr-in-rutile temperature calculation in [Section 4](#Section4).

The Cr and Nb values are transformed around Cr = 5*(Nb-500), so that above 1 is mafic and below 1 is pelitic. This calculation is performed in [Section 3.1](#Section3). The absolute values along y-axis do not have much meaning, but the plots provide a sense for how protoliths are distributed across date.


In [ ]:
### Plot Mafic-Pelitic vs Corrected Date
# y-axis is essentially distance of point from Triebold 2012 discrimination line. Defined as 'MPValue_Triebold2012' in Section 3 cell 1.
# no uncertainty filter applied

filter1 = ~( (DateCorr == 0) | DateCorr.isna() | df['MPClassification'].isna() ) # filter out values = 0 or NaN

# plot M-P by date
plt.close()
fig, axes = plt.subplots(nrows=3, figsize=(8, 15))
# Define the colors based on the mafic-pelitic classification of Triebold et al. 2012
#MPcolors = ['green' if y < 0 else 'orange' for y in MPValue_Triebold2012[filter1]]
# Scatter plot with specified colors
#plt.scatter(DateCorr[filter1], (line_Triebold2012/Cr)[filter1], c=MPcolors, edgecolors = "black", linewidth = 0.5)

# Scatter plot for mafic points
axes[0].scatter(KohnTempC[(df['MPClassification'] == 'mafic')], (line_Triebold2012/Cr)[(df['MPClassification'] == 'mafic')],
                label='Mafic', s=25, c='green', edgecolors='black', linewidth=0.5, zorder=5)

# Scatter plot for pelitic points
axes[0].scatter(KohnTempC[(df['MPClassification'] == 'pelitic')], (line_Triebold2012/Cr)[(df['MPClassification'] == 'pelitic')],
                label='Pelitic', s=25, c='orange', edgecolors='black', linewidth=0.5, zorder=4)


axes[0].axhline(y=1, color='black', linestyle='--', linewidth=1) # moderate vs high Temp line
axes[0].set_yscale('log')
axes[0].set_xlim(300, 900)
axes[0].set_xlabel('Temperature (°C)')
axes[0].set_ylabel('Mafic vs. Pelitic')
axes[0].set_title('Protolith vs Temperature')


# Protolith vs Temperature for points with U-Pb
# Scatter plot for mafic points
axes[1].scatter(KohnTempC[(filter1) & (df['MPClassification'] == 'mafic')], (line_Triebold2012/Cr)[(filter1) & (df['MPClassification'] == 'mafic')],
                label='Mafic', s=25, c='green', edgecolors='black', linewidth=0.5, zorder=5)

# Scatter plot for pelitic points
axes[1].scatter(KohnTempC[(filter1) & (df['MPClassification'] == 'pelitic')], (line_Triebold2012/Cr)[(filter1) & (df['MPClassification'] == 'pelitic')],
                label='Pelitic', s=25, c='orange', edgecolors='black', linewidth=0.5, zorder=4)


axes[1].axhline(y=1, color='black', linestyle='--', linewidth=1) # moderate vs high Temp line
axes[1].set_yscale('log')
axes[1].set_xlim(300, 900)
axes[1].set_xlabel('Temperature (°C)')
axes[1].set_ylabel('Mafic vs. Pelitic')
axes[1].set_title(f'Protolith vs Temperature (only points with TREE and U-Pb)')

# Protolith vs Temperature for points with U-Pb below uncertainty threshold
# Scatter plot for mafic points
axes[2].scatter(KohnTempC[(mask) & (filter1) & (df['MPClassification'] == 'mafic')], (line_Triebold2012/Cr)[(mask) & (filter1) & (df['MPClassification'] == 'mafic')],
                label='Mafic', s=25, c='green', edgecolors='black', linewidth=0.5, zorder=5)

# Scatter plot for pelitic points
axes[2].scatter(KohnTempC[(mask) & (filter1) & (df['MPClassification'] == 'pelitic')], (line_Triebold2012/Cr)[(mask) & (filter1) & (df['MPClassification'] == 'pelitic')],
                label='Pelitic', s=25, c='orange', edgecolors='black', linewidth=0.5, zorder=4)


axes[2].axhline(y=1, color='black', linestyle='--', linewidth=1) # moderate vs high Temp line
axes[2].set_yscale('log')
axes[2].set_xlim(300, 900)
axes[2].set_xlabel('Temperature (°C)')
axes[2].set_ylabel('Mafic vs. Pelitic')
axes[2].set_title(f'Protolith vs Temperature (only points with TREE and U-Pb <{uncertainty_threshold}% uncertainty)')

plt.show()

In [ ]:
### Run to save figure

user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "ProtolithvsTemp_UPb_v1.pdf"),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        fig.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

<a id='Section4'></a>
# 4. Zr-in-Rutile Thermometry

The following section uses the zirconium concentration data to explore rutile temperature. Temperatures are calculated for 3 different Zr-in-rutile formulations (Zack et al., 2004; Watson et al., 2006; Kohn, 2020). The temperatures calculated with the Kohn (2020) equation are used in subsequent plots.

Update the Zr concentration variable with the correct column name in the DataFrame ('Zr' in [Section 1](#DefineVariables)).

## 4.1 Calculate Zr-in-rutile temperature

In [ ]:
### Initial Zr-in-rutile temperature calculations
# Run once
# Appends variabes to DataFrame

Pressure = 13000 # pressure in bars, Storey and Pereira 2023 recommend 13000 bars detritals
PressureErr = 5000 # to estimate uncertainty, Storey and Pereira 2023 recommend 5000 bars
R = 8.3144 # Gas constant, R

KohnTempC = (71360+(0.378*Pressure)-(0.13*Zr))/(130.66-(R*np.log(Zr))) - 273.15 # Kohn 2020 eqn 13. in Celcius
KohnTempC_err = KohnTempC - ((71360+(0.378*(Pressure-PressureErr))-(0.13*Zr))/(130.66-(R*np.log(Zr))) - 273.15)
WatsonTempC = (4470./(7.36 - logZr))-273; # Watson et al 2006 temperature formulation. in Celcius
ZackTempC = 127.8 * logZr - 10; # Zack et al 2004 temperature formulation. in Celcius


### Write variables into DataFrame
# 1. Drop columns in DataFrame so can write new data
columns_to_drop = ['Pressure', 'PressureErr','KohnTempC','KohnTempC_err','WatsonTempC','ZackTempC', 'TClassification', 'Classification'] # Columns to drop
for column in columns_to_drop:# Check if each column exists before dropping
    if column in df.columns:
        df = df.drop(columns=[column])
        print(f"Column '{column}' dropped, to be replaced in DataFrame.")
    else:
        print(f"Column '{column}' does not exist in the DataFrame, does not need to be dropped.")

# 2. Add variables as columns to the DataFrame
df['Pressure'] = Pressure
df['PressureErr'] = PressureErr
df['KohnTempC'] = KohnTempC
df['KohnTempC_err'] = KohnTempC_err
df['TClassification'] = np.nan # Create a new column 'Tclass' and initialize with default value
df['WatsonTempC'] = WatsonTempC
df['ZackTempC'] = ZackTempC


# Classify into 'moderate' (350-600 C) and 'high' (> 600 C) temperature groups  
df.loc[(KohnTempC >= 350) & (KohnTempC <= 600), 'TClassification'] = 'moderate'
df.loc[KohnTempC > 600, 'TClassification'] = 'high'
    

print('\nFirst 10 rows of temperatures (°C) calculated with Kohn 2020 eqn:\n', KohnTempC.head(10))


In [ ]:
# Run to display the combined DataFrame
pd.set_option('display.max_columns', None)
df.head(10)  # display first 10 rows of the combined DataFrame
#df.tail(10)

<a id='Section4plot1'></a>
## 4.2 Zr-in-Rutile Temperature vs Corrected Date 
The next cell plots the Zr-in-rutile temperature on concordia diagrams (uncorrected U-Pb data). Subplot 1 is all uncorrected U-Pb data on Tera-Wasserburg diagram; subplot 2 excludes points above given uncertainty threshold set in [Section 3](#UPbUncertainty) (i.e., 20% uncertainty on 238/206 and 207/206). The concordia age markers are defined in [Section 1](#ConcordiaPoints).

Note: Axis limits are set below and should be adjusted. U-Pb analyses displayed as points not error ellipses.

In [ ]:
### Plot Zr-in-rutile temperatures on Tera-Wasserburg diagram

total_concordia = np.count_nonzero(~np.isnan(Xs386)) # number of U-Pb points not NaN

# Plot in Tera-Wasserburg space
plt.close()
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))
axes[0].scatter(X86tMyr, Y76tMyr, s=10, edgecolors='black', facecolors='black', linewidth=0.5, zorder=2)  # Mark specified points along concordia in black
axes[0].plot(X86, Y76, color='red', zorder=1)  # Plot concordia
for i, age in enumerate(tMyr / 1e6): # plot text labels on concordia
    axes[0].text(X86tMyr[i] - 0.75, Y76tMyr[i] - 0.075, f'{age:.0f}', fontsize=8, color='black', ha='right', va='bottom')

# Scatter plot of uncorrected U-Pb points
scatter1 = axes[0].scatter(Xs386, Ys76, c=KohnTempC, cmap=cm.roma_r, label=f'All uncorrected analyses\n(n={total_concordia})', edgecolors='black', linewidth=0.5, zorder=3)

plt.xlabel(r'$^{238}$U/$^{206}$Pb', fontsize=12)
plt.ylabel(r'$^{207}$Pb/$^{206}$Pb', fontsize=12)

# Add colorbar
colorbar1 = fig.colorbar(scatter1, ax=axes[0])
colorbar1.set_label('Temperature (°C)', rotation=270, labelpad=15)

# Set x and y-axis limits
axes[0].set_xlim([-5, 100])  # Replace with desired values
axes[0].set_ylim([-0.05, 2])  # Replace with desired values
axes[0].legend()
axes[0].grid(False)


# Subplot 2, uncertainty threshold applied
axes[1].scatter(X86tMyr, Y76tMyr, s=10, edgecolors='black', facecolors='black', linewidth=0.5, zorder=2)  # Mark specified points along concordia in black
axes[1].plot(X86, Y76, color='red', zorder=1)  # Plot concordia
for i, age in enumerate(tMyr / 1e6): # plot text labels on concordia
    axes[1].text(X86tMyr[i] - 0.35, Y76tMyr[i] - 0.04, f'{age:.0f}', fontsize=8, color='black', ha='right', va='bottom')

# Plot only the points below uncertainty threshold
scatter2 = axes[1].scatter(Xs386[mask], Ys76[mask], c=KohnTempC[mask], cmap=cm.roma_r,  
            label=f'Uncorrected analyses below \nuncertainty threshold ({uncertainty_threshold}%)\n(n={mask_sum}/{total_concordia})', edgecolors='black', 
            linewidth=0.5, zorder=3)

colorbar2 = fig.colorbar(scatter2, ax=axes[1])
colorbar2.set_label('Temperature (°C)', rotation=270, labelpad=15)

axes[1].set_xlim([-5, 100])  # Replace with desired values
axes[1].set_ylim([-0.05, 1.2])  # Replace with desired values
axes[1].legend()
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
### Run to save figure

user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "ZrTemp_concordia_v1.pdf"),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        fig.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

## 4.3 Zr-in-Rutile Temperature vs. Corrected Date -- Colored by Protolith 
The next cell plots the Zr-in-rutile temperature versus Pb-corrected date with points colored by mafic-pelitic classification. The corrected date variable is set as 'DateCorr' in [Section 1](#DefineVariables). The uncertainty_mask is defined in [Section 3](#UPbUncertainty) and applied here with the same conditions.

Note: X-axis limit is set below and should be adjusted


In [ ]:
# Define Pb corrected age from column in DataFrame
# Uses same uncertainty filter ('mask') as defined in above cell

# Set data filters  
filter2 = ~( (DateCorr == 0) | DateCorr.isna() | (KohnTempC == 0) | KohnTempC.isna() ) # filter out values = 0 or NaN
filter2_MPValue_Triebold2012 = MPValue_Triebold2012[filter2]
mask_MPValue_Triebold2012 = filter2_MPValue_Triebold2012[mask]
MPcolors1 = ['green' if y < 0 else 'orange' for y in filter2_MPValue_Triebold2012]
MPcolors2 = ['green' if y < 0 else 'orange' for y in mask_MPValue_Triebold2012]
print('Mafic is green, pelitic is orange')

plt.close()
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))
axes[0].scatter(DateCorr[filter2], KohnTempC[filter2], c=MPcolors1, edgecolors = "black", linewidth = 0.5)
axes[0].set_xlabel('Corrected Date (Ma)')
axes[0].set_ylabel('Zr-in-rutile Temperature (°C)')
axes[0].set_title('Zr-in-rutile Temperature vs Corrected Date')
axes[0].set_xlim(0, 1000)
axes[0].axhline(y=600, color='black', linestyle='--', linewidth=1) # moderate vs high Temp line

# Subplot uses uncertainty threshold mask defined in above concordia plots of Zr temps
axes[1].scatter(DateCorr[mask & filter2], KohnTempC[mask & filter2], c=MPcolors2, 
            label=f'Uncorrected analyses \nbelow uncertainty threshold\n(n={mask_sum}/{total_concordia})', edgecolors='black', 
            linewidth=0.5, zorder=3)
axes[1].set_xlim(0, 1000)
axes[1].axhline(y=600, color='black', linestyle='--', linewidth=1) # moderate vs high Temp line

axes[1].set_xlabel('Corrected Date (Ma)')
axes[1].set_ylabel('Zr-in-rutile Temperature (°C)')
axes[1].set_title(f'Zr-in-rutile Temperature vs Corrected Date Below Uncertainty Threshold ({uncertainty_threshold}%)')


plt.show()


In [ ]:
### Run to save figure

user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "TempVsDate_MPcolor_v1.pdf"),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        fig.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

<a id='Section5'></a>
# 5. Low U Rutile

This section explores low U rutile within the dataset. To run this section, the DataFrame must include "percent concordance" which is calculated in the 207Pb correction section of the Common Pb Correction notebook.


## 5.1 Zr-in-rutile temperature vs U concentration

In [ ]:
### Zr-in-rutile temperature vs U concentration

filter3 = ~( U.isna() | KohnTempC.isna() | df['MPClassification'].isna() ) # filter out values = NaN

# Define the colors based on the mafic-pelitic classification of Triebold et al. 2012
MPcolors3 = ['green' if y < 0 else 'orange' for y in MPValue_Triebold2012[filter3]]

plt.close()
plt.scatter(U[filter3], KohnTempC[filter3], c=MPcolors3, edgecolors='black', linewidth=0.5)
plt.xscale('log')
plt.axvline(x=4, color='black', linestyle='--', linewidth = 1, label='x=4') # Vertical line at U = 4 ppm
y1 = 500
y2 = 750
plt.axhline(y1, color='black', linestyle='--', linewidth = 1, label='y=500')
plt.axhline(y2, color='black', linestyle='--', linewidth = 1,label='y=750')
# Adding text labels just above the horizontal lines
plt.text(5e-3, y2 + 10, 'Granulite', color='black', fontsize=10, va='bottom')
plt.text(30, y1 + 10, 'Amphibolite/\nEclogite', color='black', fontsize=10, va='bottom')
plt.text(30, y1 - 75, 'Greenschist/\nBlueschist', color='black', fontsize=10, va='bottom')
plt.text(3.5, 785, '4 ppm', rotation='vertical', va='bottom', ha='right', color='black', fontsize=10)



# Adding colorbar
#cbar = plt.colorbar()
#cbar.set_label('U (ppm)')

# Labeling axes
plt.xlabel('U (ppm)')
plt.ylabel('Zr-in-rutile Temperature (°C)')

# Show the plot
plt.show()

In [ ]:
### Run to save figure

user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "ZrvsU_v1.pdf"),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        plt.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

## 5.2 Concordia Diagram Colored by U Concentration
Subplots of concordia diagrams colored by U concentration. The second subplot uses the uncertainty filter defined in [Section 3](#UPbUncertainty) and applied here with the same conditions. The concordia age markers are defined in [Section 1](#ConcordiaPoints).

Note: U-Pb analyses displayed as points not error ellipses.

In [ ]:
# Concordia diagram colored by U concentration
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker

### Plot Zr-in-rutile temperatures on Tera-Wasserburg diagram
palette = 'copper_r'#'coolwarm'

total_concordia = np.count_nonzero(~np.isnan(Xs386)) # number of U-Pb points not NaN


# Plot in Tera-Wasserburg space
plt.close()
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))
axes[0].scatter(X86tMyr, Y76tMyr, s=10, edgecolors='black', facecolors='black', linewidth=0.5, zorder=2)  # Mark specified points along concordia in black
axes[0].plot(X86, Y76, color='red', zorder=1)  # Plot concordia
for i, age in enumerate(tMyr / 1e6): # plot text labels on concordia
    axes[0].text(X86tMyr[i] - 0.75, Y76tMyr[i] - 0.075, f'{age:.0f}', fontsize=8, color='black', ha='right', va='bottom')

# Scatter plot of uncorrected U-Pb points
#norm = mcolors.LogNorm(vmin=1e-2, vmax=10000)
scatter1 = axes[0].scatter(Xs386, Ys76, c=U, cmap=palette, norm=mcolors.LogNorm(), label=f'All uncorrected analyses\n(n={total_concordia})', edgecolors='black', linewidth=0.5, zorder=3)

# Add colorbar with log scale
colorbar1 = fig.colorbar(scatter1, ax=axes[0])  
colorbar1.set_label('U (ppm)', rotation=270, labelpad=15)

# Set labels
axes[0].set_xlabel(r'$^{238}$U/$^{206}$Pb', fontsize=12)
axes[0].set_ylabel(r'$^{207}$Pb/$^{206}$Pb', fontsize=12)
axes[0].set_title(r'Tera-Wasserburg Diagram of Uncorrected U-Pb Analyses Colored by U Concentration')

# Set x and y-axis limits
axes[0].set_xlim([-5, 100])  # Replace with desired values
axes[0].set_ylim([-0.05, 2])  # Replace with desired values
axes[0].legend()
axes[0].grid(False)


# Subplot 2, uncertainty threshold applied
axes[1].scatter(X86tMyr, Y76tMyr, s=10, edgecolors='black', facecolors='black', linewidth=0.5, zorder=2)  # Mark specified points along concordia in black
axes[1].plot(X86, Y76, color='red', zorder=1)  # Plot concordia
for i, age in enumerate(tMyr / 1e6): # plot text labels on concordia
    axes[1].text(X86tMyr[i] - 0.35, Y76tMyr[i] - 0.04, f'{age:.0f}', fontsize=8, color='black', ha='right', va='bottom')

# Plot only the points below uncertainty threshold
scatter2 = axes[1].scatter(Xs386[mask], Ys76[mask], c=U[mask], cmap=palette, norm=mcolors.LogNorm(),  
            label=f'Uncorrected analyses below \nuncertainty threshold ({uncertainty_threshold}%)\n(n={mask_sum}/{total_concordia})', edgecolors='black', 
            linewidth=0.5, zorder=3)

colorbar2 = fig.colorbar(scatter2, ax=axes[1])
colorbar2.set_label('U (ppm)', rotation=270, labelpad=15)

# Set labels
axes[1].set_xlabel(r'$^{238}$U/$^{206}$Pb', fontsize=12)
axes[1].set_ylabel(r'$^{207}$Pb/$^{206}$Pb', fontsize=12)
axes[1].set_title(r'Uncertainty Threshold Applied')

# Set axis limits
axes[1].set_xlim([-5, 100])  # Replace with desired values
axes[1].set_ylim([-0.05, 1.2])  # Replace with desired values
axes[1].legend()
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
### Run to save figure

user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "TW_by_U_v1.pdf"),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        fig.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

## 5.3 Comparison of U concentration and concordance

### 5.3.1 U concentration and concordance with no U-Pb uncertainty filter
No uncertainty filter is applied to the U-Pb data (the plots are repeated below with an unceratainty filter).
The variables for percent concordance, U-Pb ratios and corrected date are defined in [Section 1](#DefineVariables). The concordia age markers are defined in [Section 1](#ConcordiaPoints).

In [ ]:
### Zr-in-rutile temperatures versus U contents
# Uses U (ppm), Percent Concordance, and Corrected Date

cmap = 'RdGy_r' 

# Determine the quadrant for each point
df['Quadrant'] = np.nan
df.loc[(U > 4) & (PercConc > 40), 'Quadrant'] = 1
df.loc[(U > 4) & (PercConc <= 40), 'Quadrant'] = 2
df.loc[(U <= 4) & (PercConc <= 40), 'Quadrant'] = 3
df.loc[(U <= 4) & (PercConc > 40), 'Quadrant'] = 4



# Subplot 1: Define quadrants based on U and Concordance 
# Scatter plot spanning two columns in the first row
plt.close()
plt.figure(figsize=(10,8))
plt.subplot(2, 2, 1)
scatter1 = plt.scatter(PercConc, U, c=df['Quadrant'], s=50, edgecolors='black', linewidth=0.5, cmap=cmap)
plt.axhline(4, color='black', linestyle='--', linewidth=1, label='U = 4 ppm')
plt.axvline(40, color='black', linestyle='--', linewidth=1, label='PercConc = 40%')
plt.xlabel('Concordance (%)')
plt.ylabel('U (ppm)')
plt.yscale('log')
plt.xlim([-3,103])


# Subplot 2
plt.subplot(2, 2, 2)

# Plot concordia diagram in Tera-Wasserburg space
plt.scatter(X86tMyr, Y76tMyr, s=10, edgecolors='black', facecolors='black', linewidth=0.5, zorder=2, label='Concordia Points')  # Mark specified points along concordia in black
plt.plot(X86, Y76, color='red', zorder=1, label='Concordia Curve')  # Plot concordia

# Mask NaN values in the Quadrant column or percent concordant values of NaN or 0.0
nan_mask = df['Quadrant'].isna() | PercConc.isna() | PercConc == 0

# Scatter plot for non-NaN values of Quadrants
plt.scatter(i386[~nan_mask], i76[~nan_mask], c=df['Quadrant'][~nan_mask], cmap=cmap, s=50, edgecolors='black', linewidth=0.5, zorder=4)

# Scatter plot for NaN values (white points with black edge)
plt.scatter(i386[nan_mask], i76[nan_mask], color='white', edgecolors='black', linewidth=0.5, zorder=3)


plt.xlabel(r'$^{238}$U/$^{206}$Pb', fontsize=12)
plt.ylabel(r'$^{207}$Pb/$^{206}$Pb', fontsize=12)
# Set x and y-axis limits for the second subplot
plt.xlim([-10, 100])  # Replace with your desired values
plt.ylim([-0.025, 1])  # Replace with your desired values


# Subplot 3: U versus Date
plt.subplot(2, 1, 2)
#scatter3 = plt.subplot2grid((2, 2), (1, 0), colspan=2)
#scatter3.scatter(DateCorr, U, c=df['Quadrant'], cmap=cmap, s=50, edgecolors='black', linewidth=0.5, zorder=3)

filtered_df = df[df['Quadrant'] != 3] # do not display points <4 ppm U and <40% concordant

plt.scatter(filtered_df['It5_**Age207c**'], filtered_df['Approx_U_PPM_mean'], c=filtered_df['Quadrant'], cmap=cmap, s=50, edgecolors='black', linewidth=0.5, zorder=3)
plt.axhline(4, color='black', linestyle='--', linewidth=1, label='U = 4 ppm')

plt.yscale('log')
plt.yscale('log')
plt.xlabel('Corrected Date (Ma)')
plt.ylabel('U (ppm)')
plt.xlim([0, 2000])

plt.tight_layout()
plt.show()

In [ ]:
### Run to save figure

user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "LowU_NoUncertFilter_v1.pdf"),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        plt.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

### 5.3.1 U concentration and concordance with U-Pb uncertainty filter applied
Uses the uncertainty threshold ('mask') defined in [Section 3](#UPbUncertainty) (i.e, 20% uncertainty).

In [ ]:
# Zr-in-rutile temperatures versus U contents

cmap = 'RdGy_r' 

# Determine the quadrant for each point
df['Quadrant'] = np.nan
df.loc[(U > 4) & (PercConc > 40), 'Quadrant'] = 1
df.loc[(U > 4) & (PercConc <= 40), 'Quadrant'] = 2
df.loc[(U <= 4) & (PercConc <= 40), 'Quadrant'] = 3
df.loc[(U <= 4) & (PercConc > 40), 'Quadrant'] = 4

print(f'Plot replicated with U-Pb uncertainty threshold ({uncertainty_threshold}%) applied to all subplots')

# Subplot 1: Define quadrants based on U and Concordance 
# Scatter plot spanning two columns in the first row
plt.close()
plt.figure(figsize=(10,8))
plt.subplot(2, 2, 1)
scatter1 = plt.scatter(PercConc[mask], U[mask], c=df['Quadrant'][mask], s=50, edgecolors='black', linewidth=0.5, cmap=cmap)
plt.axhline(4, color='black', linestyle='--', linewidth=1, label='U = 4 ppm')
plt.axvline(40, color='black', linestyle='--', linewidth=1, label='PercConc = 40%')
plt.xlabel('Concordance (%)')
plt.ylabel('U (ppm)')
plt.yscale('log')


# Subplot 2
plt.subplot(2, 2, 2)

# Plot concordia diagram in Tera-Wasserburg space
plt.scatter(X86tMyr, Y76tMyr, s=10, edgecolors='black', facecolors='black', linewidth=0.5, zorder=2, label='Concordia Points')  # Mark specified points along concordia in black
plt.plot(X86, Y76, color='red', zorder=1, label='Concordia Curve')  # Plot concordia


# Mask NaN values in the Quadrant column or percent concordant values of NaN or 0.0
nan_mask = df['Quadrant'].isna() | PercConc.isna() | PercConc == 0

# Scatter plot for non-NaN values of Quadrants
plt.scatter(i386[~nan_mask & mask], i76[~nan_mask & mask], c=df['Quadrant'][~nan_mask & mask], cmap=cmap, s=50, edgecolors='black', linewidth=0.5, zorder=4)

# Scatter plot for NaN values (white points with black edge)
plt.scatter(i386[nan_mask], i76[nan_mask], color='white', edgecolors='black', linewidth=0.5, zorder=3)


plt.xlabel(r'$^{238}$U/$^{206}$Pb', fontsize=12)
plt.ylabel(r'$^{207}$Pb/$^{206}$Pb', fontsize=12)
# Set x and y-axis limits for the second subplot
plt.xlim([-3, 100])  # Replace with your desired values
plt.ylim([0, 1])  # Replace with your desired values


# Subplot 3: U versus Date
plt.subplot(2, 1, 2)
#scatter3 = plt.subplot2grid((2, 2), (1, 0), colspan=2)
#scatter3.scatter(DateCorr, U, c=df['Quadrant'], cmap=cmap, s=50, edgecolors='black', linewidth=0.5, zorder=3)

filtered_df = df[df['Quadrant'] != 3] # do not display points <4 ppm U and <40% concordant

plt.scatter(filtered_df['It5_**Age207c**'][mask], filtered_df['Approx_U_PPM_mean'][mask], c=filtered_df['Quadrant'][mask], cmap=cmap, s=50, edgecolors='black', linewidth=0.5, zorder=3)
plt.axhline(4, color='black', linestyle='--', linewidth=1, label='U = 4 ppm')

plt.yscale('log')
plt.yscale('log')
plt.xlabel('Corrected Date (Ma)')
plt.ylabel('U (ppm)')
plt.xlim([0, 2000])

plt.tight_layout()
plt.show()

In [ ]:
### Run to save figure

user_input = input("\033[1mDo you want to save the plot? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save \n Either way, will display figure \n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, f'LowU_{uncertainty_threshold}percUncertaintyFilter_v1.pdf'),
        filetypes=["*.pdf"],
        title="Select Folder and File Name to Save PDF File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        fig.savefig(file_path, format="pdf")
        print("\033[1mFigure saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mFigure not saved.\033[0m")
else:
    print("\033[1mFigure not saved.\033[0m")

<a id='Section6'></a>
## 6. Export New DataFrame


Throughout the notebook, new columns have been added to the initial dataframe. This section exports the new dataframe.

In [ ]:
### Run to view new dataframe
# Columns added from protolith, temperature and U-concordance quadrant calcultions (far right)

df.head(10)

In [ ]:
### Run to save to Excel file

user_input = input("\033[1mDo you want to export the dataframe? (yes/no): \033[0m\n yes/no is case sensitive \n If yes, will open window to save.\n")

if user_input.lower() == 'yes':
    # Ask the user for the folder and filename to save the file
    file_path = easygui.filesavebox(
        default=os.path.join(folder_path1, "TREE_dataframe_export_v1.xlsx"),
        filetypes=["*.xlsx"],
        title="Select Folder and File Name to Save Excel File"
    )

    # If the user selected a file path, save the figure
    if file_path:
        df.to_excel(file_path, index=False)
        print("\033[1mDataFrame saved successfully at:\033[0m", file_path)
    else:
        print("\033[1mDataFrame not saved.\033[0m")
else:
    print("\033[1mDataFrame not saved.\033[0m")